In [1]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from nltk import edit_distance

In [2]:
from sklearn.metrics import mean_squared_error
import math
from model import *
from dataloading import *
import torch.nn.functional as F

def dataloader_fromfile(filename, batch_size=15):
    df = pd.read_csv(filename)
    inputs = df['Attribute_value']
    targets = df['Numerical_value']
    
    batch_size=15
    n_chunks = math.ceil(len(inputs)/BATCH_SIZE)
    
    inputs = prepare_data(inputs)
    targets = prepare_targets(targets)
    
    inputs = torch.chunk(inputs, n_chunks, dim=1) 
    targets = torch.chunk(targets, n_chunks, dim=1) 
    
    return inputs,targets

def dataloader(filenames, batch_size=15):
    inputs=[]
    targets=[]
    for filename in filenames:
        df = pd.read_csv(filename)
        inputs = df['Attribute_value']
        targets = df['Numerical_value']
        
        inputs+=list(inputs)
        targets+=list(targets)
    
    batch_size=15
    n_chunks = math.ceil(len(inputs)/BATCH_SIZE)
    
    inputs = prepare_data(inputs)
    targets = prepare_targets(targets)
    
    inputs = torch.chunk(inputs, n_chunks, dim=1) 
    targets = torch.chunk(targets, n_chunks, dim=1) 
    
    return inputs,targets

def interpret_output(outputs):
    outputs = F.softmax(outputs, dim=2)
    outputs = torch.argmax(outputs, dim=2)
    nums = []
    for j in range(outputs.size()[1]):
        seen_period=False
        num=''
        for i in range(1, outputs.size()[0]):
            #print(outputs[i][j])
            if outputs[i][j]==13:
                break
            dig = indexToDigit(outputs[i][j])
            
            #skip not valid periods and negative signs
            if dig=='-' and num!='':
                continue
            if dig=='.':
                if seen_period:
                    continue
                seen_period=True
            num+=dig
            
        #print(num)
        if num=='' or '-.':
            num='0'
        nums.append(float(num))
        #print(num)
    return nums

def evaluate(model, inputs, targets, optimizer, criterion, computing_device):
    model.eval()
    total_loss=0.0
    total_dist=0.0
    
    total_outputs=[]
    total_targets=[]
    
    
    with torch.no_grad():   
        for i in range(len(targets)):
                src = inputs[i].to(computing_device)
                trg = targets[i].to(computing_device)

                outputs = model(src, trg, teacher_forcing_ratio=0.0)
                labels = torch.argmax(trg, dim=2) # grab indices for loss function

                num_predictions = interpret_output(outputs)
                num_labels = interpret_output(trg)
                
                total_outputs+=num_predictions
                total_targets+=num_labels
                
                for i in range(len(num_predictions)):
                    dist = edit_distance(str(num_predictions[i]),str(num_labels[i]))
                    total_dist+=dist

                #outputs = outputs.to(computing_device)
                
                print('expected')
                print(trg.size())
                print(torch.argmax(trg, dim=2))
                print(outputs.size())
                print(torch.argmax(outputs, dim=2))
                
                #mse = mean_squared_error(num_labels,num_predictions)
                #total_mse+=mse
                #print('MSE',mse)
                
                outputs = outputs.view(-1, outputs.shape[-1]) 
                labels = labels.view(-1)

                loss = criterion(outputs, labels)
                total_loss+=loss.item()
                
    print(len(total_targets))
    print(len(total_outputs))
    print("TOTAL MSE", mean_squared_error(total_targets,total_outputs))
    print("TOTAL R2 SCORE", r2_score(total_targets,total_outputs))
    print("TOTAL DIST", total_dist)
    return total_loss/len(targets)

def evaluate_old(model, inputs, targets, optimizer, criterion, computing_device):
    model=model.to(computing_device)
    model.eval()
    

    for i in range(len(targets)):
            src = inputs[i].to(computing_device)
            trg = targets[i].to(computing_device)
            
            print(len(src))
            print(len(trg))

            optimizer.zero_grad()

            outputs = model(src, trg, teacher_forcing_ratio=0.0)
            outputs = outputs.to(computing_device)

            print('expected')
            print(trg.size())
            print(torch.argmax(trg, dim=2))
            print(outputs.size())
            print(torch.argmax(outputs, dim=2))

            num_labels = interpret_output(trg)

            #print(num_labels)

            num_predictions = interpret_output(outputs)

            #print(num_predictions)

            # shape = [seq_len, batch_size]
            mse = mean_squared_error(num_labels,num_predictions)
            print('MSE',mse)
            
    total_mse/=len(targets)
    print('TOTAL MSE',total_mse)
    print(len(targets))
    return total_mse

In [8]:
from utils import init_seq2seq
from dataloading import *

n_digits = 10
n_chars=256

config = {
            'epochs':100,
            'N_early_stop':30,
            'batch_size':512,
            'learning_rate':0.001,
            'weight_decay':0,
            'teacher_forcing_ratio':1.0,
            'hidden_dim':512,
            'n_layers':4, 
            'enc': {
                'hid_dropout':0.0,
                'input_dropout':0.0
            },
            'dec': {
                'hid_dropout':0.0,
                'input_dropout':0.0
            },

            'input_dim':n_chars+4,
            'output_dim':n_digits+5,

            'verbose':True
        }

computing_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = init_seq2seq(config, computing_device)
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'],weight_decay=config['weight_decay'])
criterion = nn.CrossEntropyLoss(ignore_index=output_pad_index)
#PATH = "output/hd={}_nl={}/bs=512_lr=0.001_wd=0_tf=1.0_hd=0.0_id=0.0_fold=A_best.pt".format(config['hidden_dim'],config['n_layers'])

PATH = "output/SIMPLE_hd=512_nl=2/bs=512_lr=0.001_wd=0_tf=1.0_hd=0.0_id=0.0_fold=E_best.pt"

model.load_state_dict(torch.load(PATH))
model.eval()

FileNotFoundError: [Errno 2] No such file or directory: 'output/SIMPLE_hd=512_nl=2/bs=512_lr=0.001_wd=0_tf=1.0_hd=0.0_id=0.0_fold=A_best.pt'

In [3]:
import os

data_dir = 'data/numerical_data_set_simple'

filenames = []
filenames_by_type = {'A':[], 'B':[], 'C':[], 'D':[], 'E':[]}
for file in os.listdir(data_dir):
    filename, file_extension = os.path.splitext(file)
    typ = filename[-1]
        
    if typ!='C' and ('gen' in filename or "._labelled" in filename):
        continue
    
    if typ in filenames_by_type:
        filenames.append(file)
        filenames_by_type[typ].append(file)
        
print(len(filenames))
print(filenames_by_type)
for key in filenames_by_type:
    print(len(filenames_by_type[key]))

17
{'A': ['labelled_extr_data1_A.csv', 'labelled_extr_data3_A.csv'], 'B': ['labelled_extr_data19_B.csv', 'labelled_extr_data5_B.csv', 'labelled_extr_data18_B.csv'], 'C': ['labelled_gen_data11_C.csv', 'labelled_gen_data9_C.csv', 'labelled_gen_data8_C.csv', 'labelled_gen_data10_C.csv', 'labelled_gen_data7_C.csv'], 'D': ['labelled_extr_data4_D.csv', 'labelled_extr_data2_D.csv'], 'E': ['labelled_dir_data92_E.csv', 'labelled_dir_data49_E.csv', 'labelled_dir_data91_E.csv', 'labelled_extr_data20_E.csv', 'labelled_dir_data39_E.csv']}
2
3
5
2
5


In [4]:
from utils import get_test_data
from utils import get_train_data

test_type='D'
inputs,targets = get_train_data(filenames_by_type,test_type, BATCH_SIZE=512,data_dir='data/numerical_data_set_simple')

computing_device="cpu"

In [48]:
evaluate(model, inputs, targets, optimizer, criterion, computing_device)

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 3,  3,  1,  ...,  1,  1,  1],
        [ 6,  6,  4,  ...,  8,  4, 10],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 3,  3,  1,  ...,  1,  1,  1],
        [10, 10,  5,  ...,  4,  5,  4],
        ...,
        [11, 11, 13,  ..., 11, 11, 11],
        [11, 11, 13,  ..., 11, 11, 11],
        [11, 11, 13,  ..., 11, 11, 11]])
MSE 455358708123.52234
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 7,  1,  1,  ...,  7,  1,  3],
        [ 1,  3,  4,  ...,  8,  1,  2],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 2,  1,  1,  ...,  2,  1,  3],
        [ 1, 

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 1,  1,  4,  ...,  1,  1,  1],
        [ 1,  6, 10,  ...,  0,  0,  0],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 1,  1,  4,  ...,  1,  1,  1],
        [ 1,  1,  5,  ...,  1,  4,  4],
        ...,
        [11, 11, 13,  ..., 11, 11, 11],
        [11, 11, 13,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 306957238899.6029
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 9,  1,  1,  ...,  1,  0,  8],
        [ 8,  4,  1,  ...,  1, 13,  5],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 4,  1, 11,  ...,  1,  0,  2],
        [10,  

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  5,  1,  ...,  3,  3,  2],
        [ 0,  7,  3,  ...,  1,  0,  5],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 9,  3,  1,  ...,  2,  3,  4],
        [ 1,  3,  1,  ...,  1,  3,  0],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 590475563.20592
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  3,  1,  ...,  3,  1,  3],
        [ 5,  3,  0,  ...,  1,  9,  9],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 4,  3,  1,  ...,  2,  1,  3],
        [ 0,  3,

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 1,  0,  4,  ...,  1,  7,  1],
        [ 1, 13,  3,  ...,  1,  2,  8],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 1,  0,  2,  ...,  1,  2,  1],
        [ 1, 13,  0,  ...,  1,  1,  7],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 52254349.5546875
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 0,  1,  5,  ...,  6,  1,  0],
        [13,  2,  2,  ...,  0,  0, 13],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 0,  1,  3,  ...,  4,  1,  0],
        [13,  9

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  1,  4,  ...,  0,  1,  0],
        [ 9,  5,  3,  ..., 13,  0, 13],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 2,  1,  2,  ...,  0,  1,  0],
        [ 1,  1,  3,  ..., 13,  1, 13],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 348682104.4844092
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 1,  0,  5,  ...,  3,  1,  6],
        [ 5, 13,  0,  ...,  0,  8,  4],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 1,  0,  3,  ...,  3,  1,  2],
        [ 9, 1

...

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 5,  1,  2,  ...,  7,  2,  6],
        [ 3,  4,  1,  ...,  9,  4,  3],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 3,  1,  2,  ...,  2,  2,  2],
        [ 3,  1,  1,  ...,  1,  2,  3],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 64378538129.3125
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 3,  2,  8,  ...,  1,  1,  2],
        [ 9,  3,  2,  ...,  9,  3,  6],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 3,  2,  2,  ...,  1,  1,  2],
        [ 3,  2

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  4,  7,  ...,  1,  1,  6],
        [ 0,  0,  7,  ...,  2,  2,  7],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 2,  2,  2,  ...,  1,  1,  2],
        [ 2,  3,  1,  ...,  1,  1,  1],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 69882118329.49776
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 7,  1,  1,  ...,  2,  2,  1],
        [ 6,  4,  8,  ...,  8,  4,  9],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 2,  1,  1,  ...,  2,  2,  1],
        [ 1,  

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 1,  1,  1,  ...,  5,  1,  2],
        [ 6,  3,  1,  ...,  3,  7,  4],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 1,  1,  1,  ...,  3,  1,  2],
        [ 4,  1,  1,  ...,  3,  1,  3],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 237341547471.75784
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 1,  6,  1,  ...,  1,  6,  8],
        [ 5,  6,  4,  ...,  5,  0,  6],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 1,  2,  1,  ...,  1,  2,  2],
        [ 4, 

expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  2,  1,  ...,  1,  5,  1],
        [ 1,  5,  7,  ...,  5,  6,  6],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 2,  2,  1,  ...,  1,  3,  1],
        [ 1,  2,  1,  ...,  4,  3,  1],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
MSE 262696717832.13745
expected
torch.Size([24, 512, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 5,  4,  7,  ...,  5,  1,  9],
        [ 9,  8,  3,  ...,  4,  9,  6],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 512, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [ 3,  2,  2,  ...,  3,  1,  2],
        [ 3, 

2.111236737995613

In [7]:
from utils import init_seq2seq
from dataloading import *

n_digits = 10
n_chars=256

config = {
            'epochs':100,
            'N_early_stop':30,
            'batch_size':512,
            'learning_rate':0.001,
            'weight_decay':0,
            'teacher_forcing_ratio':1.0,
            'hidden_dim':8,
            'n_layers':1, 
            'enc': {
                'hid_dropout':0.0,
                'input_dropout':0.0
            },
            'dec': {
                'hid_dropout':0.0,
                'input_dropout':0.0
            },

            'input_dim':n_chars+4,
            'output_dim':n_digits+5,

            'verbose':True
        }

test_type='D'

computing_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = init_seq2seq(config, computing_device)
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'],weight_decay=config['weight_decay'])
criterion = nn.CrossEntropyLoss(ignore_index=output_pad_index)
PATH = "output/hd={}_nl={}/bs=512_lr=0.001_wd=0_tf=1.0_hd=0.0_id=0.0_fold={}_best.pt".format(config['hidden_dim'],config['n_layers'],test_type)
model.load_state_dict(torch.load(PATH))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (rnn): LSTM(260, 8)
    (dropout): Dropout(p=0.0)
  )
  (decoder): Decoder(
    (rnn): LSTM(15, 8)
    (out): Linear(in_features=8, out_features=15, bias=True)
    (dropout): Dropout(p=0.0)
  )
)

In [6]:
evaluate(model, inputs, targets, optimizer, criterion, computing_device)

expected
torch.Size([24, 500, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 4,  4,  3,  ...,  5,  4,  5],
        [10, 10, 10,  ..., 10, 10, 10],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 500, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [10, 10, 10,  ..., 10, 10, 10],
        [10, 10, 10,  ..., 10, 10, 10],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
expected
torch.Size([24, 500, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 4,  4,  4,  ...,  4,  5,  5],
        [10, 10, 10,  ..., 10, 10, 10],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 500, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [10, 10, 10,  ..., 10, 10, 10],
        [10, 10, 10,  ..., 10, 10, 1

expected
torch.Size([24, 500, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 5,  5,  4,  ...,  5,  4,  5],
        [10, 10, 10,  ..., 10, 10, 10],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 500, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [10, 10, 10,  ..., 10, 10, 10],
        [10, 10, 10,  ..., 10, 10, 10],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
expected
torch.Size([24, 500, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 4,  5,  5,  ...,  5,  5,  4],
        [10, 10, 10,  ..., 10, 10, 10],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 500, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [10, 10, 10,  ..., 10, 10, 10],
        [10, 10, 10,  ..., 10, 10, 1

expected
torch.Size([24, 500, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  3,  2,  ...,  3,  1,  1],
        [13,  8, 13,  ..., 13,  8,  5],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 500, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [13,  1, 13,  ...,  2,  1,  1],
        [13, 13, 13,  ..., 13, 13, 13],
        ...,
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11],
        [11, 11, 11,  ..., 11, 11, 11]])
expected
torch.Size([24, 500, 15])
tensor([[12, 12, 12,  ..., 12, 12, 12],
        [ 2,  6,  3,  ...,  1,  1,  9],
        [13, 13, 13,  ..., 13,  1, 13],
        ...,
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14],
        [14, 14, 14,  ..., 14, 14, 14]])
torch.Size([24, 500, 15])
tensor([[14, 14, 14,  ..., 14, 14, 14],
        [13,  1,  2,  ..., 13,  1,  1],
        [13, 13, 13,  ..., 13, 13, 1

2.379043769836426